<a href="https://colab.research.google.com/github/lsuhpchelp/lbrnloniworkshop2019/blob/master/day3_nn_R/nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Neural Network with R
===

# Outline


*   **Install and load R packages**

*   **`nnet` package**

*   **`neuralnet` package**

# Install and load R packages

May take a while on the Colab

R packages to be installed:

In [0]:
install.packages("reshape")
install.packages("faraway")
install.packages("nnet")
install.packages("neuralnet")

Load R packages and scripts:

In [0]:
library(reshape)
library(nnet)
library(faraway)
library(neuralnet)
library(datasets)
download.file("https://gist.githubusercontent.com/fawda123/7471137/raw/466c1474d0a505ff044412703516c34f1a4684a5/nnet_plot_update.r","nnet_plot_update.r")
source("nnet_plot_update.r")

# 1. `nnet` package

## 1.1 A regression example: `ozone` data
* We apply the neural networks to the `ozone` data which was analyzed before using the `nnet` package, due to Venables and Ripley (2002). 
* `Ozone` data is included in the `faraway` package, which has 330 observations on the following 10 variables. 
> * **O3** Ozone conc., ppm, at Sandbug AFB.
> * **vh** a numeric vector
>*  **wind** wind speed
>* **humidity** a numeric vector
>* **temp** temperature
>* **ibh** inversion base height
>* **dpg** Daggett pressure gradient
>* **ibt** a numeric vector
>* **vis** visibility
>* **doy** day of the year

In [0]:
attach(ozone)
summary(ozone)

### 1.1.1 Initial fitting
* We started with just **three** variables for simplicity and fit a feed-forward neural network with **one** hidden layer containing **two** units and a linear output. 
> * Why linear output? This is a regression problem. 

The result (nnmd1) from will contain # of weights and the initial and final residual sum of squres (RSS, a.k.a. sum of squared errors of prediction (SSE)): 

In [0]:
nnmd1 <- nnet(O3~temp+ibh+ibt,ozone,size=2,linout=T)

* Here we also calculate the total sum of squares:

In [0]:
sum((O3-mean(O3))^2)

## Quiz
1. Why the total weights is 11?  

2. Is this neural network model good or not? Why?

### 1.1.2 Scaling variables
* The problems comes from the initial selection of the weights. It is hard to select the initial weights when the variables have very different scales. The solution is to rescale the data to mean zero and unit variance. 

Check standard deviation before and after the scaling:

In [0]:
apply(ozone,2,sd)
ozone2 <- scale(ozone)
apply(ozone2,2,sd)

Now let's see the new RSS:

In [0]:
nnmd1 <- nnet(O3~temp+ibh+ibt,ozone2,size=2,linout=T)

### 1.1.3 Refitting the model 100 times
* Since neural network uses random initial weights, the algorithm may not give the same results for each replication. So we try to refitting the model 100 times using different initial weights and find the best fit of these 100 attempts. 

In [0]:
bestrss <- 10000
for (i in 1:100){
set.seed(i)
 nnmd1 <- nnet(O3~temp+ibh+ibt,ozone2,size=2,linout=T,trace=F)
 if (nnmd1$value < bestrss){
 bestnn <- nnmd1
 bestrss <- nnmd1$value
 }
 }
bestnn$value

Summary of bestnn:

In [0]:
summary(bestnn)

* The criterion function has 11 weights. The notation `i2 -> h1` above, for example, refers to the link between the second input variable and the first hidden neuron. `b` refers to the bias, which takes a constant value of one. We see that there is one skip-layer connection, `b->o`, from the bias to the output. 

### 1.1.4 Visualizing the network
* `nnet` package does not provide any tool to visualize the network, but some R developer contributed the source code to do that. 

In [0]:
plot.nnet(bestnn) # plot function provided by nnet_plot_update.r

## Exercise 1

1. We can put a penalty such as weight decay to obtian a more stable fit. Let's try $\lambda$ = 0.001

Hint: refer to the `nnet` package manaul to find out the options


In [0]:
bestrss <- 10000
for (i in 1:100){
set.seed(i)
 nnmd1 <- nnet(O3~temp+ibh+ibt,ozone2,size=2,linout=T, (blanks to be filled here) , trace=F)
 if (nnmd1$value < bestrss){
 bestnn <- nnmd1
 bestrss <- nnmd1$value
 }
 }
 bestnn$value
 summary(bestnn)

We can see the best RSS is somewhat larger than before. This is expected because weight decay sacrifices some fit to the current data to obtain a more stable result. 

2. Now fit the full dataset using four hidden units sine there are more inputs. 
> 2.1 Without decay $\lambda$

In [0]:
estrss <- 10000
for (i in 1:100){
set.seed(i)
 nnmd1 <- nnet(O3~ ,    ,    ,   ,)
 if (nnmd1$value < bestrss){
 bestnn <- nnmd1
 bestrss <- nnmd1$value
 }
 }
 bestnn$value
 summary(bestnn)

>2.2 With decay $\lambda$

In [0]:

bestrss <- 10000
for (i in 1:100){
set.seed(i)
 nnmd1 <- nnet(O3 ~    ,     ,     ,     ,     )
 if (nnmd1$value < bestrss){
 bestnn <- nnmd1
 bestrss <- nnmd1$value
 }
 }
 bestnn$value
 summary(bestnn)

## 1.2 Training ozone data with the `caret` package 
* In R, there is an excellent package `caret` (classification and regression training) which contains functions to streamline the model training process.
> * Can train hundreds of models with resampling methods
> * Easy to manipulate, well documented
> * Will automatically parallelize when multiple cpu cores are registered




### 1.2.1 Install and load the `caret` library

It will take a while on the Colab

In [0]:
install.packages("caret")
library(caret)

### 1.2.2 Main function (`train`) arguments

The `train` function set up a grid search on tuning parameters for vaious of data mining methods. 
* `method` A string specifying which classification or regression model to use. See http://topepo.github.io/caret/train-models-by-tag.html
* `tuneGrid` A data frame with possible tuning values. The columns are named the same asthe tuning parameters. see http://topepo.github.io/caret/available-models.html 


In [0]:
my.grid <- expand.grid(.decay = c(0.0001, 0.001,0.01), .size = c(1, 2, 3,4))
nn.model <- train(O3~ .,ozone2,method="nnet",tuneGrid = my.grid,trace=F)

In [0]:
nn.model

In [0]:
summary(nn.model)

## 2. `neuralnet` package

## 2.1 A classification example: `Infertility` data
* We apply the neural networks to the `Infertility` data using the `neuralnet` package. 
* `Infertility` data is included in the `datasets` package, which has 248 observations on the following 8 variables. 
> * **case** case status is the response, a binary variable with 1 = case and 0 = control
> * **age** age in years of case
>*  **parity** count
>* **education** 0 = 1-5 years 1 = 6-11 years 2 = 12+ years
>* **spontaneous**  number of prior spontaneous abortions 0 = 0, 1 = 1, 2 = 2 or more
>* **induced** number of prior induced abortions 0 = 0, 1 = 1, 2 = 2 or more
>* **stratum**  a numeric vector
>* **pooled.stratum** a numeric vector 


In [0]:
library(reshape)
library(faraway)
library(neuralnet)
library(datasets)

In [0]:
str(infert)
table(infert$case)

### 2.1.1 Training Set and Test Set

* Dataset could be randomly split into two parts: training set and test set.



In [0]:
set.seed(1)
indx <- sample(1:248,size=248,replace=F)
dat1 <- infert[indx[1:200],] #train set
dat2 <- infert[indx[201:248],] #test set

### 2.1.2 Main function (`neuralnet`) arguments
* `hidden` a vector of integers specifying the number of hidden neurons (vertices) in each layer.
* `threshold` a  vector about the threshold for the partial derivatives of the errorfunction as stopping criteria.
* `algorithm` a string containing the algorithm type to calculate the neural network.
* `err.fct` the error function, which is a differentiable function that is used for the calculation of the error.  Alterna-tively, the strings ’sse’ and ’ce’ which stand for the sum of squared errors andthe cross-entropy can be used.
* `act.fct` the activation function, which is a differentiable function that is used for smoothing the result of the cross productof the covariate or neurons and the weights.  Additionally the strings, ’logistic’and ’tanh’ are possible for the logistic function and tangent hyperbolicus.
* `linear.output` logical. If act.fct should not be applied to the output neurons set linear output toTRUE, otherwise to FALSE.

### 2.1.3 Model fitting
* We started with **four** variables and fit a feed-forward neural network with **one** hidden layer containing **four** units. 
* The error function is cross-entropy (`err.fct="ce"`) since this is a classification problem. 

In [0]:
set.seed(2)
nn <- neuralnet(case~age+parity+induced+spontaneous,data=dat1,hidden=4,err.fct="ce",linear.output=FALSE)
nn

### 2.1.4 Training set result

In [0]:
out <- cbind(nn$covariate,nn$net.result[[1]])
dimnames(out) <- list(NULL,c("age","parity","induced","spontaneous","nn-output"))
head(out)

### 2.1.5 Visualizing the network
* `neuralnet` package's own `plot` function is designed for an inspection of the weights for objectsof class `nn` any tool to visualize the network, but some R developer contributed the source code to do that.

In [0]:
plot(nn, rep="best")

## Exercise 2
1. Test the accuracy of the model with the test set

1.1 Choose four variables from the test data

use `subset` function

In [0]:
temp_test <- subset(dat2, select = c("age","parity","induced","spontaneous"))
head(temp_test)

Or use indexing

In [0]:
head(dat2)  # find the indices
temp_test <- dat2[,c(2:4,6)]
head(temp_test)

1.2 `neuralnet` package's own `compute` function is designed to compute the prediction variable

In [0]:
nn.results <- compute(nn, temp_test)
head(nn.results$net.result)

1.3 Compare the predicted result to the actual result

In [0]:
results <- data.frame(actual = dat2$case, prediction = nn.results$net.result)
results

1.4 Misclassification table
Build a confusion matrix: 

In [0]:
roundedresults<-sapply(results,round,digits=0)
roundedresultsdf=data.frame(roundedresults)
attach(roundedresultsdf)
table(actual,prediction)

2. Build a two-layer model. The first layer has two hidden units while the second has three

In [0]:
set.seed(2)
nn2 <- neuralnet(case~age+parity+induced+spontaneous,data=dat1,hidden=    ,err.fct="ce",linear.output=FALSE)
plot(nn2, rep="best")

# Getting Help

* Documentation: http://www.hpc.lsu.edu/docs
* Contact us
> * Email ticket system: sys-help@loni.org
> * Telephone Help Desk: 225-578-0900

